# FASE 3: Desarrollo

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Se procede a desarrollar el producto, que consiste en la <b>aplicación <i>VuelaRandom</i></b> que se encuentra encapsulada dentro de una clase contenedora de todas las variables, librerías y funciones para el correcto funcionamiento de la misma.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Esta clase incluye la parte visual (las ventanas, etiquetas y botones) y la lógica de programación y las respectivas validaciones necesarias para el correcto funcionamiento de la aplicación.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Las librerías necesarias para el correcto funcionamiento son: <b>Tkinter</b>, <b>Selenium</b> y <b>BeautifulSoup</b>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;En la versión actual, la aplicación usa los datos obtenidos desde el <i>1 de octubre del 2020</i> hasta el <i>31 de diciembre del 2020</i> para obtener 10 vuelos aleatorios con origen y precio introducidos por el usuario y actualiza los precios en base al número de días que quiera espaciar los vuelos de ida y vuelta el usuario, mostrando el resultado en una tabla y abriendo una nueva ventana de su navegador con un mapa interactivo donde puede consultar los resultados.

In [1]:
import pandas as pd
dfVuelos = pd.read_csv("../../data/data_vuelos_completo_app.csv")
dfVuelos

,CAPTURA,FUENTE,ORIGEN,DESTINO,PRECIO,AEROLINEA,fecha_salida_ida,hora_salida_ida,fecha_llegada_ida,hora_llegada_ida,fecha_salida_vuelta,hora_salida_vuelta,fecha_llegada_vuelta,hora_llegada_vuelta,dia_semana
0,2020-09-09,Ryanair,BCN,AGP,29.98,Ryanair,2020-10-01,09:25,2020-10-01,11:20,2020-10-08,07:15,2020-10-08,08:50,3
1,2020-09-09,Ryanair,BCN,BGY,25.98,Ryanair,2020-10-01,20:15,2020-10-01,22:00,2020-10-08,22:25,2020-10-08,23:59,3
2,2020-09-09,Ryanair,BCN,BHX,29.61,Ryanair,2020-10-01,12:40,2020-10-01,14:10,2020-10-08,14:35,2020-10-08,17:45,3
3,2020-09-09,Ryanair,BCN,BRU,30.24,Ryanair,2020-10-01,06:05,2020-10-01,08:20,2020-10-08,08:55,2020-10-08,11:00,3
4,2020-09-09,Ryanair,BCN,BUD,31.24,Ryanair,2020-10-01,17:00,2020-10-01,19:40,2020-10-08,20:05,2020-10-08,22:40,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2484629,2020-09-13,vuelosbaratos,MAD,SPC,329.00,Vueling,2020-12-28,09:05,2020-12-29,08:15,2021-01-04,10:45,2021-01-04,21:55,0
2484630,2020-09-13,vuelosbaratos,MAD,SPC,329.00,Vueling,2020-12-28,09:05,2020-12-29,10:15,2021-01-04,10:45,2021-01-04,21:55,0
2484631,2020-09-13,vuelosbaratos,MAD,SPC,329.00,Vueling,2020-12-28,07:25,2020-12-28,16:30,2021-01-04,13:00,2021-01-05,18:50,0
2484632,2020-09-13,vuelosbaratos,MAD,SPC,330.00,Vueling,2020-12-28,07:45,2020-12-28,19:45,2021-01-04,13:00,2021-01-05,09:40,0


In [1]:
# !/usr/bin/env python
# -*- coding: utf-8 -*-


#*******************************************************************************************************************
#                           IMPORTACIONES NECESARIAS PARA EL CORRECTO FUNCIONAMIENTO DEL PROGRAMA                  *
#*******************************************************************************************************************
from PIL import ImageTk
from bs4 import BeautifulSoup
from datetime import *
from folium import plugins
from os import *
from queue import Queue
from random import randint
from selenium import webdriver
from time import *
from tkinter import *
from tkinter import ttk

import PIL.Image
import folium
import html
import locale
import os
import pandas as pd
import random
import requests
import shutil
import threading
import tkinter as tk
import webbrowser

locale.setlocale(locale.LC_TIME, '') #Establecemos España para el formateo de fechas

#********************************************************************************************************************
#CLASE DE LA APLICACIÓN
#********************************************************************************************************************
class VuelaRandom():
    #****************************************************************************************************************
    #Variables de la clase
    #****************************************************************************************************************
    csvVuelos = "../../data/data_vuelos_completo_app.csv"
    csvAeropuertos = "../../data/aeropuertos_data.csv"
    csvOrigenes = "../../data/origenes.csv"
    
    ventana = 0
    posx_y = 0
    canvas = None

    botonAceptar = None
    botonBuscar = None
    botonLimpiar = None

    labelOrigen = None
    labelDias = None
    labelPrecio = None
    labelProgreso = None
    labelCabecera = []
    labelDatos = []
    
    comboOrigenes = None
    entryDias = None
    entryPrecio = None
    raiz = None
    
    dfOrigenes = None
    dfVuelos = None
    dfAeropuertos = None
    dfVuelosSel = None
    
    vcmd = None
    origenes = None
    

    textoProgreso = None
    textoDias = None

    panelLogo = None
    panelEntradaDatos = None
    panelInputDatos = None
    panelTabla = None
    
    mapa = None
    mapaDibujado = None
    fig = None
    ax = None
    listaLabels = []
    totalVuelos = 0
    
    #******************************************************************************************************************
    #FUNCIÓN: __init__
    #DESCRIPCIÓN: Se encarga de generar la ventana de la aplicación.
    #******************************************************************************************************************
    def __init__(self,master1):
        self.raiz = master1
        self.vcmd = (self.raiz.register(self.validate),
                '%d', '%i', '%P', '%s', '%S', '%v', '%V', '%W')
        self.raiz.wm_iconbitmap('VRicon.ico')
        self.raiz.geometry('1000x500+10+10')
        self.raiz.configure(bg = 'white')
        self.raiz.title('Vuela Random')
        self.cargaVentana1()
    
    #********************************************************************************************************************
    #FUNCIÓN: validate
    #DESCRIPCIÓN: Se encarga de validar la tecla pulsada por el usuario, como solamente se admiten números enteros, se permite dejar un campo de texto de longitud cero o un número entero.
    #********************************************************************************************************************
    def validate(self, action, index, value_if_allowed,
                       prior_value, text, validation_type, trigger_type, widget_name):
        if len(value_if_allowed) == 0:
            return True
        if value_if_allowed:
            try:
                int(value_if_allowed)
                return True
            except ValueError:
                return False
        else:
            return False
    
    #*******************************************************************************************************************
    #FUNCIÓN: cargaDfs
    #DESCRIPCIÓN: Se encarga de cargar los datos iniciales de la aplicación.
    #*******************************************************************************************************************
    def cargaDfs(self):
        print(datetime.now().strftime("%H:%M:%S")," Iniciando carga Dataframes")
        dfOrigen = pd.read_csv(self.csvOrigenes)
        self.dfAeropuertos = pd.read_csv(self.csvAeropuertos)
        dfAux = pd.merge(dfOrigen, self.dfAeropuertos, left_on='ORIGEN', right_on="COD")
        self.origenes = list(dfAux["CIUDAD"].values)
        self.dfOrigenes = dfAux.copy()
        self.dfVuelos = pd.read_csv(self.csvVuelos)
        
        self.botonAceptar =ttk.Button(self.panelLogo, text='Continuar', command=self.cargaVentanaEntradaDatos)
        self.panelLogo.grid_rowconfigure(2,minsize=10)
        self.botonAceptar.grid(row=3,column=0,sticky=S)
        print(datetime.now().strftime("%H:%M:%S")," Finalizada carga Dataframes")
        
    #*******************************************************************************************************************
    #FUNCIÓN: callback
    #DESCRIPCIÓN: Abre en una nueva ventana la URL pasada como parámetro.
    #*******************************************************************************************************************
    def callback(self,url):
        webbrowser.open(url, new=1, autoraise=False)
        
    #****************************************************************************************************************
    #FUNCIÓN: cargaVentana1
    #DESCRIPCIÓN:    Se encarga de mostrar la ventana principal y cargar los daframes iniciales
    #****************************************************************************************************************
    def cargaVentana1(self):
        
        
        self.panelLogo = tk.Frame(self.raiz)
        self.panelLogo.grid(row=0,column=0)
        self.panelLogo.configure(bg='white')
        self.canvas = Canvas(self.panelLogo, width = 500,bd=0, highlightthickness=0, height = 397)
        self.canvas.grid(row=0,column=0, sticky=N)
        self.img = ImageTk.PhotoImage(PIL.Image.open("VuelaRandom-Logotipo.png"))      
        self.canvas.create_image(0,0, anchor=NW, image=self.img) 
        self.raiz.grid_rowconfigure(0, weight=1)
        self.raiz.grid_columnconfigure(0, weight=1)

        label = tk.Label(self.panelLogo, text="Noelia Medina | David López | Guillermo Contreras")
        label.grid(row=1,column=0)
        label.configure(bg = 'white')
        
    #****************************************************************************************************************
    #FUNCIÓN: cargaVentanaEntradaDatos
    #DESCRIPCIÓN: Se encarga de mostrar la ventana para la introducción de datos por parte del usuario.
    #****************************************************************************************************************
    def cargaVentanaEntradaDatos(self):
        self.raiz.grid_rowconfigure(0, weight=0)
        self.raiz.grid_columnconfigure(0, weight=0)
        self.panelLogo.grid_forget()
        self.panelEntradaDatos = tk.Frame(self.raiz)
        self.panelEntradaDatos.grid(row=0, column=1, sticky=N)
        self.panelInputDatos = tk.Frame(self.panelEntradaDatos)
        self.panelInputDatos.grid_columnconfigure(0,minsize=10)
        self.panelInputDatos.grid(row=0,column=1)
        self.panelEntradaDatos.grid_columnconfigure(0,minsize=10)
        
        self.labelOrigen = tk.Label(self.panelInputDatos, text = "Desde dónde")
        self.labelOrigen.grid(row=0,column=1)
        self.comboOrigenes = ttk.Combobox(self.panelInputDatos, values=self.origenes)
        self.comboOrigenes.current(0)
        self.comboOrigenes.grid(row=1,column=1)
        self.panelInputDatos.grid_columnconfigure(2,minsize=10)
        
        self.labelDias = tk.Label(self.panelInputDatos, text = "Cuántos dias")
        self.labelDias.grid(row=0,column=3)
        self.textoDias = tk.StringVar()
        self.textoDias.set("7")
        self.entryDias = ttk.Entry(self.panelInputDatos, validate = 'key', validatecommand = self.vcmd, textvariable = self.textoDias)
        self.entryDias.grid(row=1,column=3)
        self.panelInputDatos.grid_columnconfigure(4,minsize=10)
        
        self.labelPrecio = tk.Label(self.panelInputDatos, text = "Cuánto quieres pagar")
        self.labelPrecio.grid(row=0,column=5)
        self.entryPrecio = ttk.Entry(self.panelInputDatos, validate = 'key', validatecommand = self.vcmd)
        self.entryPrecio.grid(row=1,column=5)
        self.panelInputDatos.grid_columnconfigure(6,minsize=10)
        
        self.textoProgreso = tk.StringVar()
        self.textoProgreso.set("")
        self.labelProgreso = tk.Label(self.panelEntradaDatos,textvariable=self.textoProgreso)
        self.panelEntradaDatos.grid_rowconfigure(1,minsize=10)
        self.labelProgreso.grid(row=2,column=1)
        self.panelEntradaDatos.grid_columnconfigure(2,minsize=10)
        
        
        self.botonBuscar = ttk.Button(self.panelInputDatos, text='Buscar Vuelos', command=self.buscarVuelos)
        self.botonBuscar.grid(row=1,column=7)
        self.panelInputDatos.grid_columnconfigure(8,minsize=10)
        
        self.canvas2 = Canvas(self.raiz, width = 212, bd=0, highlightthickness=0, height = 158)
        self.canvas2.grid(row=3,column=0, sticky=SW)
        self.img2 = ImageTk.PhotoImage(PIL.Image.open("VuelaRandom-Logotipo-small.jpg"))      
        self.canvas2.create_image(0,0, anchor=NW, image=self.img2) 
        self.botonSalir = ttk.Button(self.raiz, text='Salir', command=self.raiz.destroy)
        self.botonSalir.grid(row=3,column=2,sticky ="SE")
        self.raiz.grid_rowconfigure(1, weight=1)
        self.raiz.grid_columnconfigure(1, weight=0)
     
    #*********************************************************************************************************************
    #FUNCIÓN: limpiarPantalla
    #DESCRIPCIÓN: Se encarga de limpiar la tabla de resultados para una correcta visualización.
    #*********************************************************************************************************************
    def limpiaPantalla(self):
        try:
            if self.panelTabla != None:
                self.panelTabla.destroy()
        except Exception as inst:
            print(inst)
        return None 
    
    #*****************************************************************************************************************
    #FUNCIÓN: precioBajo
    #DESCRIPCIÓN: Comprueba si, para el origen introducido, existe algún vuelo con un precio inferior o igual al pasado como parámetro. Si existe devuelve False y, si no existe ningún vuelo, devuelve True.
    #*****************************************************************************************************************
    def precioBajo(self,codOrigen,precio):
        
        dfPrevuelos = self.dfVuelos[(self.dfVuelos["ORIGEN"] == codOrigen) & (self.dfVuelos["PRECIO"] <= float(precio))]
        if len(dfPrevuelos) > 0:
            return False
        else:
            return True
        
    #**************************************************************************************************************
    #FUNCIÓN: buscarVuelos
    #DESCRIPCIÓN: Se encarga de validar los datos introducidos por el usuario y, si son válidos, lanza el proceso de búsqueda de vuelos.
    #**************************************************************************************************************
    def buscarVuelos(self):
        self.limpiaPantalla
        origen=''
        dias=7
        precio=0
        try:
            origen = self.comboOrigenes.get()
        except:
            self.textoProgreso.set("Origen no válido")
        try:
            dias = int(self.entryDias.get())
        except:
            dias=7
        try:
            precio = int(self.entryPrecio.get())
            self.textoProgreso.set("Buscando vuelos desde " + origen + " para " + str(dias) + " días a un precio máximo de " + str(precio) + " €")
            x=threading.Thread(target=self.generaVuelos, args=(origen,dias,precio,))
            x.start()
        except Exception as inst:
            print(inst)
            self.textoProgreso.set("¡¡¡¡Debes introducir un precio!!!!")
    
    #********************************************************************************************************************
    #FUNCIÓN: generaVuelos
    #DESCRIPCIÓN: Se encarga de seleccionar 10 vuelos aleatorios, obtener los precios y URLs actualizadas de los 10 vuelos y mostrar la tabla y el mapa con los resultados obtenidos.
    #********************************************************************************************************************
    def generaVuelos(self,origen,dias,precio):
        
        codOrigen = self.dfOrigenes[self.dfOrigenes["CIUDAD"] == origen]["COD"].iat[0]
        if self.precioBajo(codOrigen,precio):
            self.textoProgreso.set("No hemos encontrado vuelos para el presupuesto indicado")
            return False
        self.dfVuelosSel = pd.DataFrame(columns=['CAPTURA','FUENTE','ORIGEN', 'DESTINO', 'SALIDA_IDA', 'LLEGADA_IDA', 'SALIDA_VUELTA', 'LLEGADA_VUELTA', 'PRECIO', "URL", "AEROLINEA"])
        copiaDfVuelos = self.dfVuelos.copy()
        copiaDfVuelos = copiaDfVuelos[(copiaDfVuelos["ORIGEN"] == codOrigen) & (copiaDfVuelos["PRECIO"]<= precio)]
        totalVuelos = len(copiaDfVuelos)
        vacio = False
        self.totalVuelos = 0
        if totalVuelos > 10:
            totalVuelos = 10
        
        while((len(self.dfVuelosSel)<totalVuelos) and (vacio == False)):
            hilos = list()
            vuelosPre = list()
            if vacio == False:
                for i in range(totalVuelos-len(self.dfVuelosSel)):
                    fila = randint(0,len(copiaDfVuelos)-1)
                    auxVuelo = copiaDfVuelos.iloc[fila,:].copy()
                    incrementoVuelo = timedelta(days=dias)
                    auxVuelo[10] = (datetime.strptime(auxVuelo[6], '%Y-%m-%d')+incrementoVuelo).strftime('%Y-%m-%d')
                    vuelosPre.append(auxVuelo)
                    copiaDfVuelos = copiaDfVuelos.drop(copiaDfVuelos.index[fila])
                    
            if vacio == False:
                for i in vuelosPre:
                    x = threading.Thread(target=self.capturavuelo, args=(i,precio))
                    hilos.append(x)
                    x.start()

                for hilo in hilos:
                    hilo.join()
            if len(copiaDfVuelos) == 0:
                vacio = True
                print("vacio",vacio)
            
#         inicio = randint(0,len())
#         self.dfVuelosSel = self.dfVuelosSel.iloc[inicio:inicio+10,:]
        self.pintaTabla()
        print("Terminada la carga")
        self.pintaMapa()
        return None
    
    #********************************************************************************************************************
    #FUNCIÓN: capturavuelo
    #DESCRIPCIÓN: Obtiene los datos en caliente del vuelo seleccionado.
    #********************************************************************************************************************
    def capturavuelo(self,vuelo,precio):
        fuente = vuelo[1]
        datos = None
        if fuente == 'vuelosbaratos':
            print("buscando en vuelosbaratos")
            datos = self.vuelosBaratos(vuelo[2],vuelo[3],vuelo[6], vuelo[10])
        else:
            print("buscando en Ryanair")
            datos = self.GetRyanair(vuelo[2],vuelo[3],vuelo[6], vuelo[10])
        if len(datos)>0:
            print("hay datos")
            if round(float(datos["PRECIO"].iat[0]),2) <= round(float(precio),2):
                datos["PRECIO"].iat[0] = round(float(datos["PRECIO"].iat[0]),2)
                self.dfVuelosSel = self.dfVuelosSel.append(datos,ignore_index= True)
                print ("vuelo de '" + fuente + "'' encontrado y añadido")
                self.totalVuelos +=1
                self.textoProgreso.set("Encontrado(s) " +  str(self.totalVuelos) + " vuelo(s)")
            else:
                print("Vuelo de '" + fuente +"' no encontrado por exceso de precio en caliente")
        else:
            print("Vuelo no encontrado")
        return None
    
    #******************************************************************************************************************
    #FUNCIÓN: pintaTabla
    #DESCRIPCIÓN: Se encarga de pintar la tabla con los resultados obtenidos.
    #******************************************************************************************************************
    def pintaTabla(self):
        if (len(self.dfVuelosSel)> 0):
            self.panelTabla = tk.Frame(self.raiz)
            self.panelTabla.grid(row=1, column=1, sticky=N)
            self.panelLogo.grid_columnconfigure(0,minsize=300)
            self.panelLogo.grid_columnconfigure(0,minsize=300)
            self.panelLogo.grid_columnconfigure(0,minsize=300)
            self.panelLogo.grid_columnconfigure(0,minsize=300)
            self.panelLogo.grid_columnconfigure(0,minsize=100)
            self.panelLogo.grid_columnconfigure(0,minsize=300)
            tk.Label(self.panelTabla,text=" ORIGEN ", borderwidth=2, relief="groove").grid(row=0,column=0, sticky='WE')
            tk.Label(self.panelTabla,text=" DESTINO ", borderwidth=2, relief="groove").grid(row=0,column=1, sticky='WE')
            tk.Label(self.panelTabla,text=" SALIDA ", borderwidth=2, relief="groove").grid(row=0,column=2, sticky='WE')
            tk.Label(self.panelTabla,text=" REGRESO ", borderwidth=2, relief="groove").grid(row=0,column=3, sticky='WE')
            tk.Label(self.panelTabla,text=" PRECIO ", borderwidth=2, relief="groove").grid(row=0,column=4, sticky='WE')
            tk.Label(self.panelTabla,text=" COMPRAR BILLETE ", borderwidth=2, relief="groove").grid(row=0,column=5, sticky='WE')
            listaUrl = list()
            self.dfVuelosSel.sort_values(["SALIDA_IDA", "PRECIO"])
            for i in range(len(self.dfVuelosSel)):
                tk.Label(self.panelTabla,text= " " + self.dfAeropuertos[self.dfAeropuertos["COD"] == self.dfVuelosSel["ORIGEN"].iat[i]]["CIUDAD"].values[0] + " ", borderwidth=2, relief="flat").grid(row=i+1,column=0, sticky='WE')
                try:
                    tk.Label(self.panelTabla,text= " " + self.dfAeropuertos[self.dfAeropuertos["COD"] == self.dfVuelosSel["DESTINO"].iat[i]]["CIUDAD"].values[0] + " ", borderwidth=2, relief="flat").grid(row=i+1,column=1, sticky='WE')
                except:
                    tk.Label(self.panelTabla,text= " " + self.dfVuelosSel["DESTINO"].iat[i] + " ", borderwidth=2, relief="flat").grid(row=i+1,column=1, sticky='WE')
                tk.Label(self.panelTabla,text= " " + self.dfVuelosSel["SALIDA_IDA"].iat[i] + " ", borderwidth=2, relief="flat").grid(row=i+1,column=2, sticky='WE')
                tk.Label(self.panelTabla,text= " " + self.dfVuelosSel["SALIDA_VUELTA"].iat[i] + " ", borderwidth=2, relief="flat").grid(row=i+1,column=3, sticky='WE')
                tk.Label(self.panelTabla,text= " " + str(self.dfVuelosSel["PRECIO"].iat[i]) + " ", borderwidth=2, relief="flat").grid(row=i+1,column=4, sticky='WE')
                listaUrl.append(tk.Label(self.panelTabla, text=" ¡Lo quiero! ", cursor="hand2", borderwidth=2, relief="raised"))
                listaUrl[i].bind("<Button-1>", lambda e, url=self.dfVuelosSel["URL"].iat[i]:self.callback(url))
                listaUrl[i].grid(row=i+1,column=5, sticky='WE')
                self.botonLimpiar = ttk.Button(self.panelInputDatos, text='Nueva búsqueda', command=self.limpiaPantalla)
                self.botonLimpiar.grid(row=1,column=8)
        else:
            self.textoProgreso.set("No hemos encontrado vuelos para el precio indicado")

    #********************************************************************************************************************
    #FUNCIÓN: pintaMapa
    #DESCRIPCIÓN: Se encarga de generar el mapa centrado en el aeroperto de origen con los diferentes aeropuertos de destino, mostrando, para cada aeropuerto de destino, el nombre de la ciudad y el precio del vuelo.
    #********************************************************************************************************************
    def pintaMapa(self):
        try:
            if path.exists('vuelos_encontrados.html'):
                os.remove('vuelos_encontrados.html')
            if len(self.dfVuelosSel):    
                latitudOrigen = self.dfAeropuertos[self.dfAeropuertos["COD"] == self.dfVuelosSel["ORIGEN"].iat[0]]["LATITUD"].values[0]
                longitudOrigen = self.dfAeropuertos[self.dfAeropuertos["COD"] == self.dfVuelosSel["ORIGEN"].iat[0]]["LONGITUD"].values[0]
                vuelosMapa = folium.Map(location=[latitudOrigen, longitudOrigen], zoom_start=5)
                datosPrecio = plugins.MarkerCluster().add_to(vuelosMapa)
            
                for i in range(len(self.dfVuelosSel)):
                    latitud = self.dfAeropuertos[self.dfAeropuertos["COD"] == self.dfVuelosSel["DESTINO"].iat[i]]["LATITUD"].values[0]
                    longitud = self.dfAeropuertos[self.dfAeropuertos["COD"] == self.dfVuelosSel["DESTINO"].iat[i]]["LONGITUD"].values[0]
                    precio = self.dfVuelosSel["PRECIO"].iat[i]
                    fecha = self.dfVuelosSel["SALIDA_IDA"].iat[i]
                    ciudad = self.dfAeropuertos[self.dfAeropuertos["COD"] == self.dfVuelosSel["DESTINO"].iat[i]]["CIUDAD"].values[0]
                    datosPrecio.add_child(folium.Marker([latitud, longitud],popup=ciudad + " " + str(fecha) + " " + str(precio) + " €")).add_to(vuelosMapa)

                datosPrecio.add_child(folium.CircleMarker([latitudOrigen, longitudOrigen],radius=7,color="red")).add_to(vuelosMapa)
                vuelosMapa.save("vuelos_encontrados.html")
                self.callback("vuelos_encontrados.html")
                
        except Exception as inst:
            print(inst)

    #****************************************************************************************************************
    #FUNCIÓN: getRyanair
    #DESCRIPCIÓN: Obtiene los datos actualizados del vuelo capturado en Ryanair.
    #****************************************************************************************************************
    def GetRyanair(self,origen, destino, fecha_ida, fecha_vuelta):
        #Driver de chrome 
        chrome_driver = "chromedriver.exe"

        #************************************************************************************************************
        #FUNCIÓN: GetRyanair.GetVuelos
        #DESCRIPCIÓN: Se encarga de obtener la información de los vuelos de Ryanair
        #************************************************************************************************************
        def GetVuelos(soup, url, origen,destino, fecha_ida):
            dfAux = pd.DataFrame(columns=['CAPTURA', 'FUENTE', 'ORIGEN', 'DESTINO', 'SALIDA', 'LLEGADA', 'PRECIO', "URL", "AEROLINEA"])
            listaVuelos = soup.find_all("div", class_='card-wrapper')
            for i in range(len(listaVuelos)):
                vuelo = listaVuelos[i]
                hora = vuelo.find('div', {'data-ref':'flight-segment.departure'}).find('span', class_='h2').text.strip()
                hora_llegada = vuelo.find('div', {'data-ref':'flight-segment.arrival'}).find('span', class_='h2').text.strip()
                salida = fecha_ida +' '+ hora
                llegada = fecha_ida +' '+ hora_llegada
                try:
                    precio = float(vuelo.find('span', class_='price-value h2 text-700').text.replace('€','').replace(',','.').strip())
                except:
                    precio = float(vuelo.find('span', class_='price-value h2 text-700 price-value--discounted').text.replace('€','').replace(',','.').strip())
                dfAux = dfAux.append({"CAPTURA": datetime.now().strftime('%Y-%m-%d'), "FUENTE": "Ryanair", "ORIGEN":origen, "DESTINO":destino, "SALIDA":salida, "LLEGADA":llegada,"PRECIO":precio, "URL":url, "AEROLINEA":"Ryanair"}, ignore_index = True)
            return dfAux
        #**********************************************************************************************************
        #     FIN FUNCIÓN
        #**********************************************************************************************************
        
        df_vuelos = pd.DataFrame(columns=['CAPTURA', 'FUENTE', 'ORIGEN', 'DESTINO', 'SALIDA_IDA', 'LLEGADA_IDA', 'SALIDA_VUELTA', 'LLEGADA_VUELTA','PRECIO', "URL", "AEROLINEA"])

        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument("--incognito")
        chrome_options.add_argument("--headless")
        navegador = webdriver.Chrome(executable_path=chrome_driver,options=chrome_options)

        df_ida = pd.DataFrame(columns=['CAPTURA', 'FUENTE', 'ORIGEN', 'DESTINO', 'SALIDA', 'LLEGADA', 'PRECIO', "URL", "AEROLINEA"])
        df_vuelta = pd.DataFrame(columns=['CAPTURA', 'FUENTE', 'ORIGEN', 'DESTINO', 'SALIDA', 'LLEGADA', 'PRECIO', "URL", "AEROLINEA"])
        try:
            url = 'https://www.ryanair.com/es/es/trip/flights/select?adults=1&teens=0&children=0&infants=0&dateOut='+fecha_ida+'&dateIn='+fecha_vuelta+'&originIata='+origen+'&destinationIata='+destino+'&isConnectedFlight=true&isReturn=true&discount=0&promoCode=&tpAdults=1&tpTeens=0&tpChildren=0&tpInfants=0&tpStartDate='+fecha_ida+'&tpEndDate='+fecha_vuelta+'&tpOriginIata='+origen+'&tpDestinationIata='+destino+'&tpIsConnectedFlight=true&tpIsReturn=true&tpDiscount=0&tpPromoCode='
            navegador.set_page_load_timeout(20)
            navegador.get(url) 
            sleep(randint(3,6))
            source = navegador.page_source
            soup = BeautifulSoup(source)
            soup2 = soup.find("button", {'data-ref':fecha_ida})
            soup3 = soup.find("button", {'data-ref':fecha_vuelta})
            try:
                precio1 = soup2.find("span", class_="price__integers carousel-date-price--selected").text.strip()
                precio2 = soup3.find("span", class_="price__integers carousel-date-price--selected").text.strip()
                if int(precio1) >0 and int(precio2) >0:
                    soup_ida = soup.find("div", class_="ng-tns-c31-8 ng-star-inserted")
                    soup_vuelta = soup.find("div", class_="ng-tns-c31-11 ng-star-inserted")
                    datos = GetVuelos(soup_ida,url,origen,destino,fecha_ida)
                    df_ida = df_ida.append(datos)
                    datos = GetVuelos(soup_vuelta,url,origen,destino,fecha_vuelta)
                    df_vuelta = df_vuelta.append(datos)
            except:
                try:
                    precio1 = soup2.find("span", class_="price__integers carousel-date-price--selected").text.strip()
                    precio2 = soup3.find("span", class_="price__integers carousel-date-price--selected").text.strip()
                    if int(precio1) >0 and int(precio2) >0:
                        soup_ida = soup.find("div", class_="ng-tns-c31-8 ng-star-inserted")
                        soup_vuelta = soup.find("div", class_="ng-tns-c31-11 ng-star-inserted")
                        datos = GetVuelos(soup_ida,url,origen,destino,fecha_ida)
                        df_ida = df_ida.append(datos)
                        datos = GetVuelos(soup_vuelta,url,origen,destino,fecha_vuelta)
                        df_vuelta = df_vuelta.append(datos)
                except:
                    pass
        except:
            print('No existe vuelo')
            pass
        finally:
            for fila_ida in range(len(df_ida)):
                for fila_vuelta in range(len(df_vuelta)):
                    df_vuelos = df_vuelos.append({'CAPTURA': df_ida['CAPTURA'].iat[fila_ida], "FUENTE": "Ryanair", 
                                "ORIGEN":origen, "DESTINO":destino, "SALIDA_IDA":df_ida["SALIDA"].iat[fila_ida], 
                                "LLEGADA_IDA":df_ida["LLEGADA"].iat[fila_ida],
                                "SALIDA_VUELTA":df_vuelta["SALIDA"].iat[fila_vuelta], 
                                "LLEGADA_VUELTA":df_vuelta["LLEGADA"].iat[fila_vuelta],
                                "PRECIO":df_ida["PRECIO"].iat[fila_ida]+df_vuelta["PRECIO"].iat[fila_vuelta],
                                "URL": df_ida["URL"].iat[fila_ida], "AEROLINEA":'Ryanair'},  ignore_index = True) 
        navegador.quit()
        if(len(df_vuelos)>0):
            return df_vuelos.sort_values(['PRECIO'],ascending=True).head(1)
        else:
            return df_vuelos
    

    #************************************************************************************************************************
    #FUNCIÓN: vuelosBaratos
    #DESCRIPCIÓN: Obtiene la información en caliente del vuelo pasado como parámetro.
    #************************************************************************************************************************
    def vuelosBaratos(self,origen, destino, fecha_ida, fecha_vuelta):
        #********************************************************************************************************************
        #FUNCIÓN: getDetalle
        #DESCRIPCIÓN: Obtiene el detalle del vuelo
        #********************************************************************************************************************
        def getDetalle(vuelo):
            salida_ida = ""
            llegada_ida = ""
            salida_vuelta = ""
            llegada_vuelta = ""
            aerolinea="" 
            aux = vuelo.find('table', class_='tblDetails')
            listaFilas = aux.find_all('tr')
            listaColumnas = listaFilas[0].find_all('td')
            salida_ida = datetime.strptime(listaColumnas[1].find('span', class_='groupDate').text, '%d %b %Y, %H:%M').strftime('%Y-%m-%d %H:%M')
            llegada_ida = datetime.strptime(listaColumnas[2].find('span', class_='groupDate').text, '%d %b %Y, %H:%M').strftime('%Y-%m-%d %H:%M')
            aerolinea = listaColumnas[4].find('img', class_= 'tail1')['alt']
            listaColumnas = listaFilas[1].find_all('td')
            salida_vuelta = datetime.strptime(listaColumnas[1].find('span', class_='groupDate').text, '%d %b %Y, %H:%M').strftime('%Y-%m-%d %H:%M')
            llegada_vuelta = datetime.strptime(listaColumnas[2].find('span', class_='groupDate').text, '%d %b %Y, %H:%M').strftime('%Y-%m-%d %H:%M')

            return salida_ida, llegada_ida, aerolinea, salida_vuelta, llegada_vuelta

        #**************************************************************************************************************
        #                             FIN FUNCIÓN
        #**************************************************************************************************************


        fecha_ida = datetime.strptime(fecha_ida, "%Y-%m-%d").strftime("%Y-%#m-%#d")
        fecha_vuelta = datetime.strptime(fecha_vuelta, "%Y-%m-%d").strftime("%Y-%#m-%#d")
        numErrores = 0

        chrome_driver = "chromedriver.exe"
        #Opción para abrir Chrome en modo incógnito
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument("--incognito")
        chrome_options.add_argument("--headless")
        navegador = webdriver.Chrome(executable_path=chrome_driver,options=chrome_options)

        dfuno = pd.DataFrame(columns=['CAPTURA','FUENTE','ORIGEN', 'DESTINO', 'SALIDA_IDA', 'LLEGADA_IDA', 'SALIDA_VUELTA', 'LLEGADA_VUELTA', 'PRECIO', "URL", "AEROLINEA"])
        #Recorre todas las rutas encontradas
        error = True
        url = "https://www.vuelosbaratos.es/Buscar/" + origen + "-" + destino + "/" + fecha_ida + '/' + fecha_vuelta + "/ES/"
        navegador.set_page_load_timeout(30)
        while error:
            try:
                navegador.get(url)
                sleep(randint(6,10))

                source = navegador.page_source
                soup = BeautifulSoup(source).body

                #Obtengo la información de los vuelos
                listaVuelos = soup.find_all('div', class_='boxBody')
                for vuelo in listaVuelos:
                    precio = int(vuelo.find('span', class_='priceBig').text.replace('.','').replace(',',''))
                    enlace = html.unescape(vuelo.find('a', class_="operatorName")["href"])
                    empresa = vuelo.find('a', class_="operatorName").getText()
                    salida_ida, llegada_ida, aerolinea, salida_vuelta, llegada_vuelta = getDetalle(vuelo)

                    #Añado el vuelo a mi DataFrame
                    dfuno = dfuno.append({"CAPTURA":datetime.now().strftime("%Y-%m-%d"),'FUENTE':'vuelosbaratos',"ORIGEN":origen, "DESTINO":destino, "SALIDA_IDA":salida_ida, "LLEGADA_IDA": llegada_ida, "SALIDA_VUELTA": salida_vuelta, "LLEGADA_VUELTA": llegada_vuelta, 'URL':url, 'AEROLINEA':aerolinea, 'PRECIO':precio} ,ignore_index=True)
                error = False
            except Exception as inst:
                print(inst)
                print(datetime.now().strftime("%H:%M:%S")," ==>          ERROR al intentar descargar la página. Error número " + str(numErrores))
                numErrores += 1
                if numErrores > 20:
                    print(datetime.now().strftime("%H:%M:%S")," ==> ERROR SE HA SUPERADO EL NÚMERO MÁXIMO DE ERRORES")
                sleep(10)

        navegador.quit()
        dfuno["PRECIO"] = dfuno["PRECIO"].astype ("int")
        if len(dfuno)>0:
            return dfuno.sort_values(["PRECIO"],ascending=True).head(1)
        else:
            return dfuno
#****************************************************************************************************************************
#                              FIN DE LA CLASE
#****************************************************************************************************************************

#***************************************************************************************************************************
#FUNCIÓN: main
#DESCRIPCIÓN: Se encarga de crear un objeto Tkinter y lanzar la aplicación Vuela Random.
#****************************************************************************************************************************
def main():
    raiz = tk.Tk()
    mi_app = VuelaRandom(raiz)
    x=threading.Thread(target=mi_app.cargaDfs)#, args=(1,))
    x.start()
#     mi_app.cargaDfs()
    raiz.mainloop()
    return 0

#****************************************************************************************************************************
#                         EJECUCIÓN DEL PROGRAMA
#****************************************************************************************************************************
if __name__ == '__main__':
    main()

14:37:47  Iniciando carga Dataframes
14:37:53  Finalizada carga Dataframes
buscando en vuelosbaratosbuscando en vuelosbaratos
buscando en vuelosbaratos

buscando en vuelosbaratosbuscando en vuelosbaratos
buscando en vuelosbaratos

buscando en vuelosbaratos
buscando en vuelosbaratosbuscando en vuelosbaratos

buscando en vuelosbaratos
hay datos
vuelo de 'vuelosbaratos'' encontrado y añadido
hay datos
vuelo de 'vuelosbaratos'' encontrado y añadido
hay datos
vuelo de 'vuelosbaratos'' encontrado y añadido
hay datos
vuelo de 'vuelosbaratos'' encontrado y añadido
hay datos
vuelo de 'vuelosbaratos'' encontrado y añadido
hay datos
vuelo de 'vuelosbaratos'' encontrado y añadido
hay datos
vuelo de 'vuelosbaratos'' encontrado y añadido
hay datos
vuelo de 'vuelosbaratos'' encontrado y añadido
hay datos
vuelo de 'vuelosbaratos'' encontrado y añadido


Exception in thread Thread-16:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\normal\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\ProgramData\Anaconda3\envs\normal\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-1-abd9cc8dab23>", line 333, in capturavuelo
  File "<ipython-input-1-abd9cc8dab23>", line 552, in vuelosBaratos
  File "C:\ProgramData\Anaconda3\envs\normal\lib\site-packages\selenium\webdriver\chrome\webdriver.py", line 76, in __init__
    RemoteWebDriver.__init__(
  File "C:\ProgramData\Anaconda3\envs\normal\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 157, in __init__
    self.start_session(capabilities, browser_profile)
  File "C:\ProgramData\Anaconda3\envs\normal\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 252, in start_session
    response = self.execute(Command.NEW_SESSION, parameters)
  File "C:\ProgramData\Anaconda3\env

buscando en vuelosbaratos
hay datos
vuelo de 'vuelosbaratos'' encontrado y añadido


Exception in thread Thread-18:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\normal\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\ProgramData\Anaconda3\envs\normal\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-1-abd9cc8dab23>", line 344, in capturavuelo
  File "C:\ProgramData\Anaconda3\envs\normal\lib\tkinter\__init__.py", line 365, in set
    return self._tk.globalsetvar(self._name, value)
RuntimeError: main thread is not in main loop
Exception in thread Thread-7:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\normal\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\ProgramData\Anaconda3\envs\normal\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-1-abd9cc8dab23>", line 319, in generaVuelos
  File "<ipython-input-1-abd9cc8dab23>", line 357, in pintaTabla
  File "C:\P